<a href="https://colab.research.google.com/github/jollyoli93/KnowYourLoadExactly/blob/main/YoloV8_CraneDetector_WorkingBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Set Mode for Notebook
set_mode = input()
MODE = "TEST"

if set_mode == 1:
  MODE = "TRAIN"
else:
  MODE = "TEST"



In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [ ]:
# !pip install ultralytics
# !pip install wandb
!pip install --upgrade ultralytics==8.3 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.3/881.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-hea

In [ ]:
import wandb

KeyboardInterrupt: 

8a3588b68f271aaffcd990c0b011ee1ef38fb83b

In [ ]:
wandb.login()

In [ ]:
from wandb.integration.ultralytics import add_wandb_callback

from ultralytics import YOLO

## For Extracting YOLO ZIP Files


In [ ]:
# !file /content/drive/MyDrive/Crane-Detector/First Experiment/master_project.v5-all_images_twocls.yolov8-obb.zip

In [ ]:
# import zipfile

# zip_path = "/content/drive/MyDrive/Crane-Detector/First Experiment/master_project-all_img_less_aug_2cls.yolov8-obb.zip"
# extract_path = "/content/drive/MyDrive/Crane-Detector/First Experiment"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

##

## Model Config

In [ ]:
%cd /content/drive/MyDrive/Crane-Detector/

/content/drive/MyDrive/Crane-Detector


In [ ]:
!ls

 boats.jpg	    'HighRes Experiments'   runs        weights
'First Experiment'  'Not Load Experiment'  'Test Run'


In [ ]:
project_path ="/content/drive/MyDrive/Crane-Detector/"
#YOLO("yolov8n-obb.pt")

In [ ]:
from pathlib import Path
import re

#get_best = f"{project_path}runs/obb/"

def extract_best(get_best):
    path = Path(get_best)
    dirs = [f for f in path.glob('train*') if f.is_dir()]

    if not dirs:
        return None

    # Sort by modification time
    dirs_sorted = sorted(dirs, key=lambda f: f.stat().st_mtime)
    last_dir = dirs_sorted[-1]

    # Extract digits from the stem
    match = re.search(r'\d+$', last_dir.stem)
    return str(match.group())[:-1] if match else None

In [ ]:
# path_to_best = f"{project_path}runs/obb/"
# extract_best(path_to_best)

In [ ]:
# --------------------------
# CONFIG
# --------------------------
MODE = "TRAIN"  # or "TEST"
MODEL = project_path +"weights/yolov8s-obb.pt" #use directory to prevent default YOLOv11
IMG_SIZE = 1600
RECT=True
EPOCHS = 100
PROJECT_NAME = "Crane-Detector"
DATA_PATH = "/content/drive/MyDrive/Crane-Detector/First Experiment/master_project.v6-all_img_less_aug_2cls.yolov8-obb/data.yaml"



In [ ]:
from ultralytics import YOLO
import torch
from ultralytics.nn.tasks import OBBModel

def train_model(model_name, img_size, epochs, data_path, rectangle):
    # # Add OBBModel to allowed globals for safe loading
    # with torch.serialization.safe_globals({"OBBModel": OBBModel}):
    model = YOLO(model_name)
    add_wandb_callback(model, wandb, enable_model_checkpointing=True)

    results = model.train(
        data=data_path,
        epochs=epochs,
        imgsz=img_size,
        rect=rectangle
    )

    return model

def test_model(weights_path, pro_path):
    path_to_best = f"{pro_path}runs/obb/"
    get_best = extract_best(path_to_best)
    best = f"{pro_path}runs/obb/train{get_best}/weights/best.pt"
    model = YOLO(weights_path)
    results = model.val(data=best)
    return results

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Run Training/Testing



In [ ]:
if MODE == "TEST":
  from ultralytics import YOLO
  import cv2

  # Load the OBB model
  model = YOLO('/content/drive/MyDrive/Crane-Detector/runs/obb/train27/weights/best.pt')

  # Read the image
  img_path = '/content/drive/MyDrive/Crane-Detector/First Experiment/ALL_2CLS_NOAUG/test/images/2507190850060000_jpg.rf.d723ac3a4e99a8b1f8e851b6283ae113.jpg'
  img = cv2.imread(img_path)

  # Run prediction
  results = model(img)

  # Iterate over detections
  for result in results:
      obb_boxes = result.obb.xyxy.cpu().numpy()  # Oriented bounding boxes in xyxy format
      class_ids = result.obb.cls.cpu().numpy().astype(int)

      for box, cls_id in zip(obb_boxes, class_ids):
          x1, y1, x2, y2 = map(int, box[:4])  # Get bounding box corners
          class_name = model.names[cls_id]

          # Draw rectangle
          cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
          cv2.putText(img, class_name, (x1, y1 - 5),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)



NameError: name 'MODE' is not defined

In [ ]:
if MODE == "TRAIN":
  run = wandb.init(project=PROJECT_NAME, job_type="train", config={
        "model": MODEL.split("/")[-1],
        "img_size": IMG_SIZE,
        "epochs": EPOCHS,
        "rect": RECT,
        "wandb":True
    })
  model = train_model(MODEL, IMG_SIZE, EPOCHS,DATA_PATH, RECT)

  val_results = model.val()
  # Log overall metrics
  run.log({
      "val_mAP50": val_results.box.map50,
      "val_mAP75": val_results.box.map75,
      "val_mAP50-95": val_results.box.map,
      "val_mean_precision": val_results.box.mp,
      "val_mean_recall": val_results.box.mr,
      "val_fitness": val_results.box.fitness,
  })

  # Log per-class metrics
  for i, class_idx in enumerate(val_results.box.ap_class_index):
      run.log({
          f"class_{class_idx}_precision": val_results.box.p[i],
          f"class_{class_idx}_recall": val_results.box.r[i],
          f"class_{class_idx}_f1": val_results.box.f1[i],
          f"class_{class_idx}_ap50": val_results.box.ap50[i],
          f"class_{class_idx}_ap": val_results.box.ap[i],
          f"class_{class_idx}_ap_curve": val_results.box.all_ap[i],  # Shape: (10,)
      })
  results = val_results

  run.finish()

In [ ]:
# --------------------------
# RUN
# --------------------------
results = ""
if MODE == "VAL"
  weights_path = f"/content/drive/MyDrive/Crane-Detector/runs/obb/train27/weights/best.pt"
  results = test_model(weights_path, DATA_PATH)


In [ ]:
!ls

In [ ]:
# test_model(MODEL, project_path)

In [ ]:
from IPython.display import Image, display

path_to_best = f"{project_path}runs/obb/"
get_best_idx = extract_best(path_to_best)
#show BoxF1_curve.png
display(Image(filename=f"/content/drive/MyDrive/Crane-Detector/runs/obb/train27/BoxF1_curve.png"))

# results.png
display(Image(filename=f"/content/drive/MyDrive/Crane-Detector/runs/obb/train27/results.png"))

# confusion.png
display(Image(filename=f"/content/drive/MyDrive/Crane-Detector/runs/obb/train27/confusion_matrix.png"))


In [ ]:
# # Evaluate the model's performance on the validation set
# results = model.val()

In [ ]:
import cv2

# Load the model
model = YOLO('/content/drive/MyDrive/Crane-Detector/First Experiment/runs/obb/train27/weights/best.pt')

# Read the image
img = cv2.imread('/content/drive/MyDrive/Crane-Detector/test/2502271000050000.jpg')

# Run prediction on the image
results = model.predict(img, stream=True)

# Iterate over the results
for result in results:
    boxes = result.obb.cpu().numpy()  # Get boxes on CPU in numpy format
    for box in boxes:  # Iterate over boxes
        r = box.xyxy[0].astype(int)  # Get corner points as int
        class_id = int(box.cls[0])  # Get class ID
        class_name = model.names[class_id]  # Get class name using the class ID
        print(f"Class: {class_name}, Box: {r}")  # Print class name and box coordinates
        cv2.rectangle(img, r[:2], r[2:], (0, 255, 0), 2)  # Draw boxes on the image

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.12/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.12/dist-packages/ultralytics/assets/bus.jpg: 1600x1216 (no detections), 2983.8ms


AttributeError: 'NoneType' object has no attribute 'cpu'

In [ ]:
# Display result
from google.colab.patches import cv2_imshow
cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Export Best Model



In [ ]:
!pip install --upgrade onnx onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.9/660.9 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 10.2 MB/s eta 0:00:00


In [ ]:
model = YOLO('/content/drive/MyDrive/Crane-Detector/First Experiment/runs/obb/train27/weights/best.pt')

In [ ]:
# Export to ONNX
model.export(format="onnx")


Ultralytics 8.3.0 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
YOLOv8s-obb summary (fused): 187 layers, 11,412,345 parameters, 0 gradients, 29.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Crane-Detector/First Experiment/runs/obb/train27/weights/best.pt' with input shape (1, 3, 1600, 1600) BCHW and output shape(s) (1, 7, 52500) (22.5 MB)
requirements: Ultralytics requirements ['onnxslim==0.1.34', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 271.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 155.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 199.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.8s, installed 2 packages: ['onnxslim==0.1.34', 'onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export wi

'/content/drive/MyDrive/Crane-Detector/First Experiment/runs/obb/train27/weights/best.onnx'